In [1]:
EXAMPLE_FILE = 'SpamData/01_Processing/practice_email.txt'
SPAM_1 = 'SpamData/01_Processing/spam_assassin_corpus/spam_1'
SPAM_2 = 'SpamData/01_Processing/spam_assassin_corpus/spam_2'
HAM_1 = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_1'
HAM_2 = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_2'
WORD_ID_FILE = 'SpamData/01_Processing/word_by_id.csv'
TRAINING_DATA_FILE = 'SpamData/02_Training/train_data.txt'
TESTING_DATA_FILE = 'SpamData/02_Training/test_data.txt'
VOCAB_SIZE = 2500

In [57]:
from os import walk
from os.path import join
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

In [3]:
stream = open(EXAMPLE_FILE, encoding = 'latin-1')
message = stream.read()
stream.close()

print(message)

From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002
Return-Path: <exmh-workers-admin@spamassassin.taint.org>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)
Received: from listman.spamassassin.taint.org (listman.spamassassin.taint.org [66.187.233.211]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for
    <zzzz-exmh@spamassassin.taint.org>; Thu, 22 Aug 2002 12:34:53 +0100
Received: from listman.spamassassin.taint.org (localhost.localdomain [127.0.0.1]) by
    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002
    07:35:02 -0400 (EDT)
Delivered-To: exmh-workers@listman.spamassassin.taint.org
Received: from int-mx1.corp

In [4]:
stream = open(EXAMPLE_FILE, encoding = 'latin-1')
is_body = False
lines = []
for line in stream:
    if is_body:
        lines.append(line)
    elif line == '\n':
        is_body = True

stream.close()

message = '\n'.join(lines)
print(message)



Dear Mr Still



Good tidings to you and all your staff for the festive season ahead (Christmas).

Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.

But WAIT! You're probably thinking: "What makes him so special?"

Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.

You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.

I look forward to hearing from you.



Best wishes

Robin Cooper

[Excerpt from the book: The Timewaster Let

In [5]:
def generate_emails(path):
    for root, dirnames, filenames in walk(path):
        for file_name in filenames:
            
            file_path = join(root, file_name)
            stream = open(file_path, encoding = 'latin-1')
            is_body = False
            lines = []
            for line in stream:
                if is_body:
                    lines.append(line)
                elif line == '\n':
                    is_body = True

            stream.close()

            message = '\n'.join(lines)
            yield file_name, message


In [6]:
def df_from_directory(path, classification):
    rows = []
    row_names = []
    
    for file_name, email_body in generate_emails(path):
        rows.append({'message' : email_body, 'category' : classification})
        row_names.append(file_name)
    
    return pd.DataFrame(rows, index = row_names)
            

In [7]:
spam_emails = df_from_directory(SPAM_1, 1)
spam_emails.head()

,category,message
00001.7848dde101aa985090474a91ec93fcf0,1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
00002.d94f1b97e48ed3b553b3508d116e6a09,1,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
00003.2ee33bc6eacdb11f38d052c44819ba6c,1,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
00004.eac8de8d759b7e74154f142194282724,1,##############################################...
00005.57696a39d7d84318ce497886896bf90d,1,I thought you might like these:\n\n1) Slim Dow...


In [8]:
spam_emails.shape

(501, 2)

In [9]:
spam_emails = spam_emails.append(df_from_directory(SPAM_2, 1))
spam_emails.shape

(1898, 2)

In [10]:
ham_emails = df_from_directory(HAM_1, 0)
ham_emails = ham_emails.append(df_from_directory(HAM_2, 0))
ham_emails.shape

(3901, 2)

In [11]:
data = pd.concat([spam_emails, ham_emails])
data.shape

(5799, 2)

In [12]:
data['message'].isnull().values.any()

False

In [13]:
(data['message'].str.len() == 0).sum()

3

In [14]:
data[data['message'].str.len() == 0]

,category,message
cmds,1,
cmds,1,
cmds,0,


In [15]:
data.index.get_loc('cmds')

array([False, False, False, ..., False, False, False])

In [16]:
data.drop(['cmds'], inplace = True)

In [17]:
(data['message'].str.len() == 0).sum()

0

In [18]:
document_ids = range(0, len(data.index))
data['doc_ids'] = document_ids

In [19]:
data['file_name'] = data.index
data.set_index('doc_ids', inplace = True)

In [20]:
data

,category,message,file_name
doc_ids,,,
0,1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",00001.7848dde101aa985090474a91ec93fcf0
1,1,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,00002.d94f1b97e48ed3b553b3508d116e6a09
2,1,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,00003.2ee33bc6eacdb11f38d052c44819ba6c
3,1,##############################################...,00004.eac8de8d759b7e74154f142194282724
4,1,I thought you might like these:\n\n1) Slim Dow...,00005.57696a39d7d84318ce497886896bf90d
5,1,A POWERHOUSE GIFTING PROGRAM You Don't Want To...,00006.5ab5620d3d7c6c0db76234556a16f6c1
6,1,Help wanted. We are a 14 year old fortune 500...,00007.d8521faf753ff9ee989122f6816f87d7
7,1,<html>\n\n<head>\n\n<title>ReliaQuote - Save U...,00008.dfd941deb10f5eed78b1594b131c9266
8,1,TIRED OF THE BULL OUT THERE?\n\nWant To Stop L...,00009.027bf6e0b0c4ab34db3ce0ea4bf2edab


In [21]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [22]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [23]:
stop_words = set(stopwords.words('english'))
msg = 'All Work and No Play makes Jack a dull boy.'
words = word_tokenize(msg.lower())

#stemming
stemmer = PorterStemmer()

filtered_words = []

for word in words:
    if word not in stop_words and word.isalpha():
        stemmed_word = stemmer.stem(word)
        filtered_words.append(stemmed_word)
        

print(filtered_words)

['work', 'play', 'make', 'jack', 'dull', 'boy']


In [24]:
'p'.isalpha()

True

In [25]:
def clean_message(message, stemmer = PorterStemmer(), stop_words = set(stopwords.words('english')) ):
    
    soup = BeautifulSoup(message, 'html.parser')
    cleaned_text = soup.get_text()
    words = word_tokenize(cleaned_text.lower())
    
    filtered_words = []

    for word in words:
        if word not in stop_words and word.isalpha():
            stemmed_word = stemmer.stem(word)
            filtered_words.append(stemmed_word)

    return filtered_words

In [26]:
clean_message(data.at[2, 'message'])

['fight',
 'risk',
 'cancer',
 'http',
 'slim',
 'guarante',
 'lose',
 'lb',
 'day',
 'http',
 'get',
 'child',
 'support',
 'deserv',
 'free',
 'legal',
 'advic',
 'http',
 'join',
 'web',
 'fastest',
 'grow',
 'singl',
 'commun',
 'http',
 'start',
 'privat',
 'photo',
 'album',
 'onlin',
 'http',
 'wonder',
 'day',
 'offer',
 'manag',
 'prizemama',
 'wish',
 'leav',
 'list',
 'pleas',
 'use',
 'link',
 'http',
 'zzzz']

In [27]:
%%time
nested_list = data.message.apply(clean_message)

C:\Users\muskan\Anaconda3\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.post-gazette.com/columnists/20020905brian5
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Wall time: 26.6 s


In [28]:
flat_list = [item for sublist in nested_list for item in sublist]
normal_words = pd.Series(flat_list).value_counts()
normal_words.shape[0]

27302

In [29]:
normal_words
frequent_words = normal_words[:2500]

In [30]:
#Generating Vocabulary Dataframe

word_ids = list(range(0, VOCAB_SIZE))
vocab = pd.DataFrame({'Vocab_word' : frequent_words.index.values}, index  = word_ids)
vocab.index.name = 'Word_ids'
vocab

,Vocab_word
Word_ids,
0,http
1,use
2,list
3,email
4,get
5,mail
6,one
7,free
8,time


In [32]:
vocab.to_csv(WORD_ID_FILE, index_label = vocab.index.name, header = vocab.Vocab_word.name)
vocab

,Vocab_word
Word_ids,
0,http
1,use
2,list
3,email
4,get
5,mail
6,one
7,free
8,time


In [33]:
nested_list

doc_ids
0       [save, life, insur, spend, life, quot, save, g...
1       [fight, risk, cancer, http, slim, guarante, lo...
2       [fight, risk, cancer, http, slim, guarante, lo...
3       [adult, club, offer, free, membership, instant...
4       [thought, might, like, slim, guarante, lose, l...
5       [powerhous, gift, program, want, miss, get, fo...
6       [help, want, year, old, fortun, compani, grow,...
7       [reliaquot, save, life, insur, life, chang, in...
8       [tire, bull, want, stop, lose, money, want, re...
9       [dear, cost, effect, direct, email, advertis, ...
10      [cellular, phone, accessori, wholesal, price, ...
11      [click, simpli, amateur, like, girl, next, doo...
12      [fight, risk, cancer, http, slim, guarante, lo...
13      [free, motorola, cell, phone, cash, back, phon...
14      [lowest, rate, servic, conferenc, easi, cent, ...
15      [want, watch, sport, event, movi, assembl, ele...
16      [help, want, year, old, fortun, compani, grow,...
17    

In [36]:
word_columns_df = pd.DataFrame.from_records(nested_list.tolist())

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(word_columns_df, data.category, test_size = 0.3, random_state = 42)


,0,1,2,3,4,5,6,7,8,9,...,7655,7656,7657,7658,7659,7660,7661,7662,7663,7664
4844,ye,inde,agent,directori,verita,cd,unix,subdirectori,file,call,...,None,None,None,None,None,None,None,None,None,None
4727,problem,come,tri,instal,harddissssk,like,alreadi,mount,http,yahoo,...,None,None,None,None,None,None,None,None,None,None
5022,origin,messag,date,mon,aug,chad,norwood,sven,cc,subject,...,None,None,None,None,None,None,None,None,None,None
3504,inlin,folk,sever,major,internet,outag,morn,across,major,provid,...,None,None,None,None,None,None,None,None,None,None
3921,url,http,date,bath,chronicl,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [38]:
X_train.tail()

,0,1,2,3,4,5,6,7,8,9,...,7655,7656,7657,7658,7659,7660,7661,7662,7663,7664
3772,url,http,date,tue,sep,gmt,wire,urgent,busi,proposit,...,None,None,None,None,None,None,None,None,None,None
5191,wander,uppsala,sweden,last,week,ran,across,keramik,studio,paula,...,None,None,None,None,None,None,None,None,None,None
5226,pm,win,trees,wrote,actual,easi,test,experiment,compar,result,...,None,None,None,None,None,None,None,None,None,None
5390,r,russel,turpin,write,r,hmm,live,life,hous,r,...,None,None,None,None,None,None,None,None,None,None
860,see,exit,inform,dear,friend,discov,five,internet,busi,set,...,None,None,None,None,None,None,None,None,None,None


In [43]:
word_index = pd.Index(vocab.Vocab_word)
word_index.get_loc('welch')

1923

In [48]:
def sparse_matrix(df, indexed_words, label):
    
    dict_list = []
    nr = df.shape[0]
    nc = df.shape[1]
    word_set = set(indexed_words)
    
    for i in range(nr):
        for j in range(nc):
            word = df.iat[i, j]
            if word in word_set:
                doc_id = df.index[i]
                category = label.at[doc_id]
                word_id = indexed_words.get_loc(word)
                
                item = {'LABEL' : category, 'DOC_ID' : doc_id, 'OCCURENCE' : 1, 'WORD_ID' : word_id}
                
                dict_list.append(item)
                
        
    return pd.DataFrame(dict_list)



In [50]:
%%time
sparse_train_df = sparse_matrix(X_train, word_index, Y_train)

Wall time: 7min 36s


In [52]:
sparse_train_df

,DOC_ID,LABEL,OCCURENCE,WORD_ID
0,4844,0,1,266
1,4844,0,1,1258
2,4844,0,1,507
3,4844,0,1,310
4,4844,0,1,255
5,4844,0,1,883
6,4844,0,1,32
7,4844,0,1,49
8,4844,0,1,24
9,4844,0,1,114


In [54]:
train_grouped = sparse_train_df.groupby(['DOC_ID', 'WORD_ID', 'LABEL']).sum()
train_grouped = train_grouped.reset_index()

In [55]:
train_grouped

,DOC_ID,WORD_ID,LABEL,OCCURENCE
0,0,2,1,1
1,0,3,1,2
2,0,4,1,1
3,0,7,1,3
4,0,11,1,1
5,0,15,1,1
6,0,18,1,2
7,0,19,1,2
8,0,23,1,1
9,0,26,1,2


In [58]:
np.savetxt(TRAINING_DATA_FILE, train_grouped, fmt = '%d')

In [59]:
%%time
sparse_test_df = sparse_matrix(X_test, word_index, Y_test)

Wall time: 3min 8s


In [60]:
test_grouped = sparse_test_df.groupby(['DOC_ID', 'WORD_ID', 'LABEL']).sum()
test_grouped = test_grouped.reset_index()

In [62]:
np.savetxt(TESTING_DATA_FILE, test_grouped, fmt = '%d')